In [ ]:
!pip install -U pandasql plotly

In [ ]:
!conda install -c plotly python-kaleido plotly-orca -y

In [1]:
import pandas as pd
from os import listdir
from pandasql import sqldf
import matplotlib as plt
import plotly.express as px

In [2]:
#definindo o diretorio
directory = 'data/'
files = listdir(directory)

In [3]:
#selecionando apenas as variaveis importantes para esta analise
col_names = ['vendor_id', 'pickup_datetime','passenger_count', 'trip_distance','payment_type', 'tip_amount', 'total_amount']

#setando os tipos para as variaveis, para que o pandas não precise adivinhar os dados e consumir mais RAM
dtypes = {
    "vendor_id": "category",
    "pickup_datetime": "datetime64",
    "passenger_count": "int",
    "trip_distance": "float",
    "payment_type": "category",
    "tip_amount":"float",
    "total_amount": "float"
}

#criando um dataframe vazio apenas para as colunas. Este dataframe servira para concatenar todos os datasets
#posteriormente
df = pd.DataFrame(columns=col_names)

#um for que le apenas os .json
for file_name in files:
    if '.json' in file_name:
        #le o primeiro json tranformando-o em dataframe do pandas
        df_new = pd.read_json(directory+file_name, lines=True, dtype=dtypes)
        #pega apenas as colunas que serao importantes para as analises e sobreescreve o dataframe
        df_new = df_new[col_names]

        #converte os tipos de pagamentos em letras minusculas
        df_new['payment_type'] = df_new['payment_type'].apply(lambda string: string.lower())

        #concatena o dataframe antigo com o novo
        df = pd.concat([df, df_new])

df = df.reset_index(drop=True)

In [ ]:
#verificando missing data
print(df.isna().sum())
print("\n -------------- \n")
print(df.isnull().sum())

In [ ]:
#sqldf para usar o pandas sql
pysqldf = lambda q: sqldf(q, globals())

## 1 - Distância média percorrida por viagens com no máximo 2 passageiros

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [ ]:
#pegando corridas com ate 2 passageiros juntamente com a distancia percorrida
df_max_2_passenger = df[df['passenger_count'] <= 2]['trip_distance']

In [ ]:
#verificando a presença de outliers
#Ao verificar a presença de outliers com boxplot, percebemos que a maioria dos dados 
#esta entre 0 e 10
plt.boxplot(df_max_2_passenger)

In [ ]:
#calculando o z-score para fazer o tratamento de outliers
z = np.abs(stats.zscore(df_max_2_passenger))

In [ ]:
#usando threshold de 3 para pegar index de outliers abaixo de z-score 3
normal_indexes = np.where(z < 3)[0]

In [ ]:
#se pegarmos o percentil 99 podemos observar que o valor esta muito proximo do que vimos no bloxplot
#o que quer dizer nossos outliers realmente sairam do dataset
df_max_2_passenger.iloc[normal_indexes].quantile(.99)

In [ ]:
#aqui conseguimos ver que o valor maximo ja na esta tao alto, o que valida mais uma vez nossa
#remocao de outliers. vale notar tambem que o max esta muito proximo do percentil 99
df_max_2_passenger.iloc[normal_indexes].max()

In [ ]:
df_max_2_passenger = df_max_2_passenger.iloc[normal_indexes]

In [ ]:
#pysqldf("SELECT avg(trip_distance) trip_distance FROM df where passenger_count <= 2;")
#multiplicando por 1.60 para obtermos o valor em kilometros temos a media da distancia percorrida
#com no maximo 2 passageiros
df_max_2_passenger.mean() * 1.609344

In [ ]:
#analisando a diferença entre 1 e 2 passageiros
df_1_passenger = df[df['passenger_count'] == 1]['trip_distance']
z = np.abs(stats.zscore(df_1_passenger))
normal_indexes = np.where(z < 3)[0]
df_1_passenger.iloc[normal_indexes].mean() * 1.609344

### 2 - Os 3 maiores vendors em quantidade total de dinheiro arrecadado

In [ ]:
#media de dinheiro arrecadado
df['total_amount'].mean()

In [4]:
#por questoes de performance, esta linha esta comentada, mas tambem pode ser excutado com pandasql
#top3_vendor = pysqldf("SELECT ROUND(sum(total_amount), 2) total_amount, vendor_id FROM df GROUP BY vendor_id ORDER BY total_amount DESC LIMIT 3;")
top3_vendor = df[['vendor_id', 'total_amount']].groupby(['vendor_id']).sum().round().sort_values(by='total_amount', ascending=False).reset_index().head(3)

In [18]:
#criando barplot
fig = px.bar(
    top3_vendor, 
    x='vendor_id', 
    y='total_amount', 
    title="3 maiores vendors em quantidade total de dinheiro arrecadado",
    text='total_amount',
    color=['CMT', 'VTS', 'DDS'],
    color_discrete_map = {'CMT': '#ffda99', 'VTS': '#99fffa', 'DDS': '#a16efa'}
)

#definindo as labels para os eixos x e y
fig.update_layout(
    xaxis_title="Vendor",
    yaxis_title="Total Arrecadado",
    legend_title="Vendor",
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=650
)

#cria os texto fora das barras
fig.update_traces(
    texttemplate='%{text:.2s}', 
    textposition='outside',
    hovertemplate='Total Arrecadado: %{y:.4s} <br> Vendor: %{x}',
)
fig.show()
#fig.write_image("graficos/top3_vendor.jpeg")

### 3 - Histograma da distribuição mensal de corridas pagas em dinheiro por ano

In [12]:
#transformando os indexes em datetime para facilitar o trabalho com datas
df.index = pd.DatetimeIndex(df['pickup_datetime'])

#reordena o dataframe em ordem crescente
df = df.sort_index()

In [ ]:
#porcentagem de cada tipo de pagamento
df.groupby('payment_type')["vendor_id"].count().apply(lambda x: (x / df.shape[0]) * 100)

In [14]:
#plotando histogramas por ano
for ano in [2009, 2010,2011,2012]:
    dataHist = df[(df['payment_type'] == 'cash') & (df.index.year == ano)]['pickup_datetime'].dt.month
    
    fig = px.histogram(dataHist, 
                   title=f'Distribuição mensal de corridas pagas em dinheiro no ano de {ano}',
                   nbins=12,
                  )
    fig.update_layout(
        showlegend=False,
        xaxis_title="Meses",
        yaxis_title="Quantidade",
        title={
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
    )
    fig.update_traces(
        hovertemplate='Quantidade: %{y:.4s} <br> Mes: %{x}',
    )
    fig.show()
    #fig.write_image(f'graficos/hist_{ano}.jpeg')

In [ ]:
#histograma de distribuicao unico para os 4 anos
dataHist = df[df['payment_type'] == 'cash']['pickup_datetime'].dt.month

In [ ]:
fig = px.histogram(dataHist, 
                   color=dataHist.index.year, 
                   title='Distribuição mensal de corridas pagas em dinheiro por ano',
                   color_discrete_map = {
                       '2009': '#ffda99', 
                       '2010': '#99fffa', 
                       '2011': '#e0ff99',
                       '2012': '#b199ff',
                   },
                   nbins=12,
                  )
fig.update_layout(
    xaxis_title="Meses",
    yaxis_title="Quantidade",
    legend_title="Ano",
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
)
fig.update_traces(
    hovertemplate='Quantidade: %{y:.4s} <br> Mes: %{x}',
)
fig.show()

### 4 - Serie temporal da quantidade de gorjetas de cada dia, nos ultimos 3 meses de 2012

In [15]:
#Pegando os ultimos 3 meses de 2012 disponiveis no dataset, nao possuimos os meses de Novembro e dezembro no
#dataset, portanto, teremos que pegar valores de Agosto - Outubro
last_3_months = df[(df.index.year == 2012) & (df['tip_amount'] > 0)].last("3M")['tip_amount'].reset_index()

#contando a quantidade de gorjetas recebidas em cada dia
last_3_months = last_3_months.groupby(last_3_months['pickup_datetime'].dt.date).size()

In [ ]:
fig = px.line(
    df, 
    x=last_3_months.index, 
    y=last_3_months,
    title='Quantidade de gorjetas de cada dia, dos ultimos 3 meses de 2012',
)
fig.update_layout(
    xaxis_title="Data",
    yaxis_title="Quantidade de gorjetas",
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=650
)
fig.update_traces(
    hovertemplate='Quantidade: %{y:.4s} <br> Data: %{x}',
)
fig.show()
fig.write_image("graficos/time_series.jpeg")